In [ ]:
using BasisFunctions, FrameFun, Plots, DomainSets, LinearAlgebra
gr()

# Introduction 

Using frames, the behaviour of the coefficients is not predictable. This is easily seen comparing a Chebyshev extension approximation with a Chebyshev series approximation.

In [ ]:
CE = FrameFun.ExtensionFramePlatform(ModelPlatform(ChebyshevT(1,-2,2)), -1..1)
CS = ChebyshevPlatform()

We approximate $f(x)=e^x$ on `0..1`,

In [ ]:
f = exp;

Using two approximation sizes

In [ ]:
N1 = 20; N2 = 40;

and see that,  for a basis,  coefficient size goes down to machine precision, for a frame, coefficients go down; but they do not reach a platform. 

In [ ]:
plot(abs.(coefficients(Fun(f, CS, N1))).+eps(),yscale=:log10,layout=2,ylims=[1e-16,10],label="Small",title="Series",size=(900,300),linestyle=:dash,marker=:dot)
scatter!(abs.(coefficients(Fun(f, CS, N2))).+eps(),yscale=:log10,label="Large",legend=:topright,linestyle=:dash)
plot!(abs.(coefficients(Fun(f, CE, N1))).+eps(),yscale=:log10,subplot=2,ylims=[1e-16,10],label="Small",title="Extension",linestyle=:dash,marker=:dot)
scatter!(abs.(coefficients(Fun(f, CE, N2))).+eps(),yscale=:log10,subplot=2,label="Large",legend=:bottomleft)

# Smooting

In this section we approximate the exponential function with a Fourier extension

In [ ]:
f = exp
P = FourierExtensionPlatform(0..0.5)

Smoothing penalizes the high frequencies. In stead of solving `Ax=b` in a least squares sense, we solve $AD^{-1}y=b$ and $x=D^{-1}y$. Such that $\|Dx\|$ is minimized while solving `Ax=b`. D is here a diagonal operator where diagonal entries depend on the frequency. The dependency of the weights on the frequency is given by

In [ ]:
function scaling(dict, idx)
    l = abs(value(native_index(dict, idx)))
    1e-4+l+l^2
end;

We solve the approximation problem two times. One in the ordinary least squares sense and one penalizing the high frequencies.

In [ ]:
F = Fun(f, P, 61; solverstyle=AZStyle())
FS = Fun(f, P, 61; solverstyle=AZSmoothStyle(), scaling=scaling)

In [ ]:
scatter(;layout=(1,3),size=(900,300))
plot!(F,subplot=1,label="regular",plot_ext=true,legend=:topleft)
plot!(FS,subplot=1,label="smooth",plot_ext=true)
plot!(F,f,subplot=2,label="regular",plot_ext=true,legend=:topleft)
plot!(FS,f,subplot=2,label="smooth",plot_ext=true)
scatter!(abs.(coefficients(F)), yscale=:log10,subplot=3,legend=false)
scatter!(abs.(coefficients(FS)), yscale=:log10,subplot=3)

Again, using more degrees of freedom does not lead to a smallest coefficient size that is smaller $\mathcal O(10^{-9})$

In [ ]:
F = Fun(f, P, 101; solverstyle=AZStyle())
FS = Fun(f, P, 101; solverstyle=AZSmoothStyle(),scaling=scaling)

In [ ]:
scatter(;layout=(1,3),size=(900,300))
plot!(F,subplot=1,label="regular",plot_ext=true,legend=:topleft)
plot!(FS,subplot=1,label="smooth",plot_ext=true)
plot!(F,f,subplot=2,label="regular",plot_ext=true,legend=:topleft)
plot!(FS,f,subplot=2,label="smooth",plot_ext=true)
scatter!(abs.(coefficients(F)), yscale=:log10,subplot=3,legend=false)
scatter!(abs.(coefficients(FS)), yscale=:log10,subplot=3)

# Adaptive approximation

We study the behaviour of adaptive frame approximation using platforms. 

## 1. Function with singularity just outside the interval of interest

We choose to approximate a function with a singularity to the right of the interval of interest (`0.0..0.5`). 

In [ ]:
f = x->.1/(x-.5-1e-1);

The function is approximated using a Fourier Extension platform. 

In [ ]:
P = FourierExtensionPlatform(0.0..0.5);

We use different stopping criteria and evaluate the properties of the resulting approximations. 

In [ ]:
ResidualStyle(), FNAStyle()

### a) Using the ResidualStyle

The `ResidualStyle()` `criterion` measures the norm of the residual of the system Ax=B; and compares it with a tolerance `tol`. 

In [ ]:
FRES = Fun(f, P, criterion = ResidualStyle(), threshold=1e-10,verbose=true)

### b) Using the FNAStyle

The `FNAStyle()` `criterion` checks two subcriteria. Firstly, it measures the norm of the coefficients. The $\ell^2$-norm of the coefficients should be smaller than `FNAeta`*$\|f\|$. Secondly, we measure the residual of the system Ax=B is measured; and compare it with a tolerance `tol` (as in `ResidualStyle()`).  

In [ ]:
FFNA = Fun(f, P, criterion = FNAStyle(), FNAeta=5, threshold=1e-10,verbose=true)

### Comparison

`FNAStyle` is more restrictive than `ResidualStyle`; therefore, it is no suprise that `FNAStyle` gives an approximation with more degrees of freedom. 

In [ ]:
length(FRES), length(FFNA) 

At the upside, coefficients are smaller. This gives a more robust approximation. Here this is visible by the size of the extension of the approximation. The extension is a lot larger using `ResidualStyle`.  

In [ ]:
plot(FFNA; plot_ext=true,label="FNA",layout=2,size=(900,300))
plot!(FRES; plot_ext=true,label="RES",legend=:topleft)
plot!(FFNA,f; plot_ext=true,label="FNA",subplot=2)
plot!(FRES,f; plot_ext=true,label="RES",legend=:topleft,subplot=2)

## 2. Function with spike and spike weighted sum/enriched frame with spike

Does not work, function is too difficult, or results similar

In [ ]:
# f = x-> sqrt(abs(sin(x)-.1)) + cos(20x);
f = x-> sqrt(sin(x)) + sin(20x);

In [ ]:
plot(f, LinRange(0,.5,100);legend=false, size=[400,200])

In [ ]:
using Calculus
struct SQRT <: Function 
(::SQRT)(x::T) where T<:Number = sqrt(x)
end

struct DIFFSQRT <: Function 
end
Calculus.derivative(::SQRT) = DIFFSQRT()
Calculus.derivative(::SQRT, x::T) where T<:Number = (DIFFSQRT())(x)
(::DIFFSQRT)(x::T) where T<:Number = one(T)/(2sqrt(x))

In [ ]:
WP = WeightedSumPlatform(P, [SQRT(),x->1])

In [ ]:
FRES = Fun(f, WP, criterion = ResidualStyle(), threshold=1e-12,verbose=true,oversamplingfactor=5)

In [ ]:
plot(FRES, f; layout=2)
plot!(FRES; plot_ext=true, subplot=2)

In [ ]:
FFNA = Fun(f, WP, criterion = FNAStyle(), FNAeta=2, threshold=1e-12,verbose=true, maxlength=1<<14,oversamplingfactor=5)

### Comparison

In [ ]:
length(FRES), length(FFNA)

In [ ]:
plot(FFNA; plot_ext=true,label="FNA",layout=2,size=(900,300))
plot!(FRES; plot_ext=true,label="RES",legend=:topleft)
plot!(FFNA,f; plot_ext=true,label="FNA",subplot=2)
plot!(FRES,f; plot_ext=true,label="RES",legend=:topleft,subplot=2)